In [11]:
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers.legacy import SGD

In [12]:
df = pd.read_csv('all_songs_and_lyrics.csv')
# Tokenize the lyrics
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
total_words = len(tokenizer.word_index) + 1
max_sequence_length = max(len(seq) for seq in tokenizer.texts_to_sequences(df['text']))

In [14]:
sequences = tokenizer.texts_to_sequences(df['text'])
text_data = pad_sequences(sequences, maxlen=max_sequence_length)

# Other non-text features
non_text_data = df[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']].values

# Labels: The next word in the sequence for each word in the lyrics
labels = np.roll(text_data, shift=-1, axis=-1)
labels[:, -1] = 0  # Replace the last element with 0 to represent padding

In [31]:
# Assuming 'text' is the user-specified text input, and 'features' are the non-textual features
text_input = Input(shape=(max_sequence_length,), name='text_input')
text_embedding = Embedding(input_dim=total_words, output_dim=100, input_length=max_sequence_length)(text_input)
lstm_output = LSTM(100)(text_embedding)

non_text_input = Input(shape=(11,), name='non_text_input')
combined_features = Concatenate()([lstm_output, non_text_input])

# Output layer for text generation
output = Dense(total_words, activation='softmax')(combined_features)

# Create the model
model = Model(inputs=[text_input, non_text_input], outputs=output)

In [33]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [34]:
model.fit({'text_input': text_data, 'non_text_input': non_text_data}, labels, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/domidanke/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/domidanke/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/Users/domidanke/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/Users/domidanke/anaconda3/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/Users/domidanke/anaconda3/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/Users/domidanke/anaconda3/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/Users/domidanke/ana